In [1]:
#!/usr/bin/env python
# coding: utf-8

import tensorflow as tf

import os, sys, getopt
import json
from datetime import datetime

import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.densenet import DenseNet121, DenseNet169, DenseNet201
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.layers import Dense, Input, GlobalAveragePooling2D
from tensorflow.keras import Model
from pathlib import Path

print(tf.__version__)

2.1.0


In [2]:
import handshape_datasets as hd

def load(dataset_name):
    """
    Load ciarp dataset.

    Returns (x, y): as dataset x and y.

    """
    dataset_path = os.path.join('develop','data',dataset_name,'data')

    if not os.path.exists(dataset_path):
        os.makedirs(dataset_path)
    data = hd.load(dataset_name, Path(dataset_path))

    return data[0], data[1]['y']


In [3]:
def train(dataset_name = "lsa16", rotation_range = 10, width_shift_range = 0.10,
          height_shift_range = 0.10, horizontal_flip = True,
          model_type='DenseNet121', lr = 0.001, epochs = 10,
          max_patience = 25, batch_size= 16, checkpoints = False, weight_classes = False,
          train_size=None, test_size=None):

    # log
    log_freq = 1
    save_freq = 40
    models_directory = 'models/'
    results_directory = 'results/'
    config_directory = 'config/'

    general_directory = Path.cwd() / "results/"
    save_directory = general_directory / "{}/transfer-learning/".format(dataset_name)
    results = 'epoch,loss,accuracy,test_loss,test_accuracy\n'

    date = datetime.now().strftime("%Y_%m_%d-%H:%M:%S")
    identifier = "densenet_transfer_learning-{}".format(
        dataset_name) + date

    summary_file = general_directory / 'summary.csv'

    # create summary file if not exists
    if not summary_file.exists():
        file = open(summary_file, 'w')
        file.write("datetime, model, config, min_loss, min_loss_accuracy\n")
        file.close()

    print("hyperparameters set")
    #print(tf.test.is_gpu_available())

    x, y = load(dataset_name)

    image_shape = np.shape(x)[1:]

    x_train, x_test, y_train, y_test = train_test_split(x,
                                                        y,
                                                        train_size=train_size,
                                                        test_size=test_size,
                                                        random_state=42,
                                                        stratify=y)
    x_train, x_test = x_train / 255.0, x_test / 255.0

    n_classes = len(np.unique(y))

    if weight_classes:
        class_weights = compute_class_weight('balanced', 
                                            np.unique(y),
                                            y)
    
    print("data loaded")

    datagen = ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization=True,
        rotation_range=rotation_range,
        width_shift_range=width_shift_range,
        height_shift_range=height_shift_range,
        horizontal_flip=horizontal_flip,
        fill_mode='constant',
        cval=0)
    datagen.fit(x_train)

    test_datagen = ImageDataGenerator(
        featurewise_center=True,
        featurewise_std_normalization=True,
        fill_mode='constant',
        cval=0)
    test_datagen.fit(x_train)

    # get the model
    if (model_type == 'DenseNet121'):
        base_model = DenseNet121(include_top=False, weights='imagenet', input_shape=image_shape)
    elif (model_type == 'DenseNet169'):
        base_model = DenseNet169(include_top=False, weights='imagenet', input_shape=image_shape)
    elif (model_type == 'DenseNet201'):
        base_model = DenseNet201(include_top=False, weights='imagenet', input_shape=image_shape)
    else: 
        print('Error: Wrong model type (DenseNet121,DenseNet169,DenseNet201)')
        return

    base_model.trainable = False
    global_average_layer = GlobalAveragePooling2D()
    hidden_dense_layer = Dense(1024, activation='relu')
    prediction_layer = Dense(n_classes, activation='softmax')
    model = tf.keras.Sequential([
        base_model,
        global_average_layer,
        hidden_dense_layer,
        prediction_layer
    ])
    
    print("model created")

    if weight_classes:
        loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

        def weightedLoss(originalLossFunc, weightsList):

            @tf.function
            def lossFunc(true, pred):

                axis = -1 #if channels last 
                #axis=  1 #if channels first

                #argmax returns the index of the element with the greatest value
                #done in the class axis, it returns the class index    
                classSelectors = tf.argmax(true, axis=axis, output_type=tf.int32) 

                #considering weights are ordered by class, for each class
                #true(1) if the class index is equal to the weight index   
                classSelectors = [tf.equal(i, classSelectors) for i in range(len(weightsList))]

                #casting boolean to float for calculations  
                #each tensor in the list contains 1 where ground true class is equal to its index 
                #if you sum all these, you will get a tensor full of ones. 
                classSelectors = [tf.cast(x, tf.float32) for x in classSelectors]

                #for each of the selections above, multiply their respective weight
                weights = [sel * w for sel,w in zip(classSelectors, weightsList)] 

                #sums all the selections
                #result is a tensor with the respective weight for each element in predictions
                weightMultiplier = weights[0]
                for i in range(1, len(weights)):
                    weightMultiplier = weightMultiplier + weights[i]


                #make sure your originalLossFunc only collapses the class axis
                #you need the other axes intact to multiply the weights tensor
                loss = originalLossFunc(true,pred) 
                loss = loss * weightMultiplier

                return loss
            return lossFunc
        loss_object = weightedLoss(loss_object, class_weights)
    else:
        loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

    optimizer = tf.keras.optimizers.Adam()

    # compile the model (should be done *after* setting layers to non-trainable)
    model.compile(optimizer=optimizer, loss=loss_object)

    train_loss = tf.keras.metrics.Mean(name='train_loss')
    train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

    test_loss = tf.keras.metrics.Mean(name='test_loss')
    test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

    @tf.function
    def train_step(images, labels):
        with tf.GradientTape() as tape:
            predictions = model(tf.cast(images, tf.float32), training=True)
            loss = loss_object(labels, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        train_loss(loss)
        train_accuracy(labels, predictions)

    @tf.function
    def test_step(images, labels):
        predictions = model(tf.cast(images, tf.float32), training=False)
        t_loss = loss_object(labels, predictions)

        test_loss(t_loss)
        test_accuracy(labels, predictions)

    # create summary writers
    train_summary_writer = tf.summary.create_file_writer(save_directory / ('summaries/train/' + identifier))
    test_summary_writer = tf.summary.create_file_writer(save_directory /  ('summaries/test/' + identifier))

    # create data generators
    train_gen =  datagen.flow(x_train, y_train, batch_size=batch_size)
    test_gen = test_datagen.flow(x_test, y_test, batch_size=batch_size, shuffle=False)

    print("starting training")

    min_loss = 100
    min_loss_acc = 0
    patience = 0

    for epoch in range(epochs):
        batches = 0
        for images, labels in train_gen:
            train_step(images, labels)
            batches += 1
            if batches >= len(x_train) / 32:
                # we need to break the loop by hand because
                # the generator loops indefinitely
                break

        batches = 0
        for test_images, test_labels in test_gen:
            test_step(test_images, test_labels)
            batches += 1
            if batches >= len(x_test) / 32:
                # we need to break the loop by hand because
                # the generator loops indefinitely
                break

        if (epoch % log_freq == 0):
            results += '{},{},{},{},{}\n'.format(epoch,
                                train_loss.result(),
                                train_accuracy.result()*100,
                                test_loss.result(),
                                test_accuracy.result()*100)
            print ('Epoch: {}, Train Loss: {}, Train Acc:{}, Test Loss: {}, Test Acc: {}'.format(epoch,
                                train_loss.result(),
                                train_accuracy.result()*100,
                                test_loss.result(),
                                test_accuracy.result()*100))

            if (test_loss.result() < min_loss):    
                if not (save_directory / models_directory).exists():
                    os.makedirs(save_directory / models_directory)
                # serialize weights to HDF5
                model.save_weights(save_directory / (models_directory + "best{}.h5".format(identifier)))
                min_loss = test_loss.result()
                min_loss_acc = test_accuracy.result()
                patience = 0
            else:
                patience += 1

            with train_summary_writer.as_default():
                tf.summary.scalar('loss', train_loss.result(), step=epoch)
                tf.summary.scalar('accuracy', train_accuracy.result(), step=epoch)
                train_loss.reset_states()           
                train_accuracy.reset_states()           

            with test_summary_writer.as_default():
                tf.summary.scalar('loss', test_loss.result(), step=epoch)
                tf.summary.scalar('accuracy', test_accuracy.result(), step=epoch)
                test_loss.reset_states()           
                test_accuracy.reset_states()   
                
        if checkpoints and epoch % save_freq == 0:
            if not (save_directory / models_directory).exists():
                os.makedirs(save_directory / models_directory)
            # serialize weights to HDF5
            model.save_weights(save_directory / (models_directory+"{}_epoch{}.h5".format(identifier,epoch)))
            
        if patience >= max_patience:
            break

    if not (save_directory / results_directory).exists():
        os.makedirs(save_directory / results_directory)
    file = open(save_directory / (results_directory + 'results-'+ identifier + '.csv'),'w') 
    file.write(results) 
    file.close()

    if not (save_directory / config_directory).exists():
        os.makedirs(save_directory / config_directory)

    config = {
        'data.dataset_name': dataset_name, 
        'data.rotation_range': rotation_range, 
        'data.width_shift_range': width_shift_range, 
        'data.height_shift_range': height_shift_range, 
        'data.horizontal_flip': horizontal_flip, 
        'model.reduction': reduction, 
        'train.lr': lr, 
        'train.epochs': epochs, 
        'train.max_patience': max_patience, 
        'train.batch_size': batch_size, 
    }

    file = open(save_directory / (config_directory + identifier + '.json'), 'w')
    file.write(json.dumps(config, indent=2))
    file.close()

    file = open(summary_file, 'a+')
    summary = "{}, {}, densenet-tranfer-learning, {}, {}, {}\n".format(date,
                                                       dataset_name,
                                                       save_directory / (config_directory + identifier + '.json'),
                                                       min_loss,
                                                       min_loss_acc)
    file.write(summary)
    file.close()


In [4]:
train()

hyperparameters set
data loaded
model created


ValueError: Attempt to convert a value (WindowsPath('C:/Users/okaso/OneDrive/Escritorio/tesis/Rios-Gaston-tesis-2020/src/transfer_learning/results/lsa16/transfer-learning/summaries/train/densenet_transfer_learning-lsa162020_02_05-21:48:03')) with an unsupported type (<class 'pathlib.WindowsPath'>) to a Tensor.

In [ ]:
'''
# create the base pre-trained model
base_model = InceptionV3(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='relu')(x)
# and a logistic layer -- let's say we have 200 classes
predictions = Dense(200, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional InceptionV3 layers
for layer in base_model.layers:
    layer.trainable = False

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# train the model on the new data for a few epochs
model.fit_generator(...)

# at this point, the top layers are well trained and we can start fine-tuning
# convolutional layers from inception V3. We will freeze the bottom N layers
# and train the remaining top layers.

# let's visualize layer names and layer indices to see how many layers
# we should freeze:
for i, layer in enumerate(base_model.layers):
   print(i, layer.name)

# we chose to train the top 2 inception blocks, i.e. we will freeze
# the first 249 layers and unfreeze the rest:
for layer in model.layers[:249]:
   layer.trainable = False
for layer in model.layers[249:]:
   layer.trainable = True

# we need to recompile the model for these modifications to take effect
# we use SGD with a low learning rate
from keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy')

# we train our model again (this time fine-tuning the top 2 inception blocks
# alongside the top Dense layers
model.fit_generator(...)
'''